# Notebook

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import neat
import matplotlib.pyplot as plt

import pickle
import multimodal_mazes

path = '../Results/test2/'

In [ ]:
# Load config data 
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                        neat.DefaultSpeciesSet, neat.DefaultStagnation,
                        path + 'neat_config.ini')

exp_config = multimodal_mazes.load_exp_config(path + 'exp_config.ini')
print('Maze noise: ' + str(exp_config['maze_noise_scale']))
print('Sensor noise: ' + str(exp_config['sensor_noise_scale']))

In [ ]:
# Load data 
x = np.load(path + '99.npy')

top_agent = np.where(x['fitness'] == x['fitness'].max())

with open(path + '99.pickle', 'rb') as file:
    genomes = pickle.load(file)

genome_id, genome, channels = genomes[top_agent[0][0]]

print(x[top_agent[0][0]])
print(genome.size())

In [ ]:
# WIP: hidden layers 

# n = 4692 # 4692
# genome_id, genome, channels = genomes[n]

In [ ]:
# Plotting

# Fitness 
multimodal_mazes.plot_fitness_over_generations(x, plot_species=True)

# Architecture
node_names = {-1: 'Ch0 L', -2: 'Ch1 L', -3 : 'Ch0 R', -4 : 'Ch1 R', 
              -5: 'Ch0 U', -6: 'Ch1 U', -7 : 'Ch0 D', -8 : 'Ch1 D',
              0 : 'Act L', 1 : 'Act R', 2 : 'Act U', 3 : 'Act D'}
genome = multimodal_mazes.prune_architecture(genome, config)
plt.figure()
multimodal_mazes.plot_architecture(genome, config, node_names=node_names)

In [ ]:
# Top genomes 
fitness_cutoff = .95

# Generate mazes
maze = multimodal_mazes.TrackMaze(size=exp_config['maze_size'], n_channels=len(exp_config['channels']))
maze.generate(exp_config['n_mazes'], noise_scale=exp_config['maze_noise_scale'])

# Test agents 
top_agents = [] 
for g in np.where(x['fitness'] >= fitness_cutoff)[0]: 
    _, genome, channels = genomes[g]
    genome = multimodal_mazes.prune_architecture(genome, config)
    fitness = multimodal_mazes.eval_fitness(genome, config, channels, exp_config['sensor_noise_scale'], maze, exp_config['n_steps'])
    if fitness >= fitness_cutoff: 
        top_agents.append(g)
print(len(top_agents))

In [ ]:
# Architecture metrics 

top_agents_metrics_n, top_agents_metrics_p = [], []
for g in top_agents:
    _, genome, channels = genomes[g]
    genome = multimodal_mazes.prune_architecture(genome, config)
    metrics_n, metrics_p = multimodal_mazes.architecture_metrics(genome, config, channels)
    top_agents_metrics_n.append(list(metrics_n.values()))
    top_agents_metrics_p.append(list(metrics_p.values()))
        
top_agents_metrics_n = np.array(top_agents_metrics_n)
top_agents_metrics_p = np.array(top_agents_metrics_p)

# Plotting 
f, (a0, a1) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [0.4, 0.6]}, figsize=(15, 5))
a0.plot(top_agents_metrics_n.T, c='k', linewidth=1.5, alpha=0.25);
a0.set_xticks(np.arange(len(metrics_n)), list(metrics_n.keys()), rotation='vertical')
a0.set_ylabel('Number')

a1.plot(top_agents_metrics_p.T, c='k', linewidth=1.5, alpha=0.25);
a1.set_xticks(np.arange(len(metrics_p)), list(metrics_p.keys()), rotation='vertical')
a1.set_ylabel('Value')
a1.set_ylim([-0.025, 1.025])

In [ ]:
# Robustness to sensor noise
noise_scales = np.linspace(start=0.0, stop=0.5, num=10)
noise_results, noise_baseline = multimodal_mazes.robustness_to_sensor_noise(agents=top_agents, noise_scales=noise_scales, n_mazes=100, exp_config=exp_config, genomes=genomes, config=config)
multimodal_mazes.plot_robustness(condition_values=noise_scales, condition_results=noise_results, condition_label='Sensor noise', noise_baseline=noise_baseline, agents=top_agents, genomes=genomes, config=config, node_names=node_names)